In [1]:
import re
import os

def load_tex_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file]

def process_line(line):
    return re.sub(r'(\$.*?\$|\\\[.*?\\\]|\\\(.*?\\\))', '', line)

def process_corpus(corpus):
    return [process_line(line) for line in corpus]

def find_tex_files(root_folder):
    tex_files = []
    for root, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.tex'):
                tex_files.append(os.path.join(root, file))
    return tex_files

def process_all_tex_files(root_folder):
    all_processed_corpus = []
    tex_files = find_tex_files(root_folder)
    
    for filepath in tex_files:
        corpus = load_tex_file(filepath)
        processed_corpus = process_corpus(corpus)
        all_processed_corpus.extend(processed_corpus)
    
    return all_processed_corpus

# 사용 예시
root_folder = r'C:\Users\wjdrb\Downloads\drive-download-20240719T063242Z-001'
processed_corpus = process_all_tex_files(root_folder)


In [6]:
corpus = ["F(x)={\sqrt{\frac{10x^{-8}}{-8}}}+C_{1} =-{\frac{5}{4x^{8}}}+C_{1}\quad{\mathrm {if~}}x<0.",
"F(x)=\sqrt{10}x^{-8/-8}+C_{1}=-\frac{5} {4}x^{8}+C_{1}\quad\mathrm{if~}x<0."]


<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\wjdrb\AppData\Local\Temp\ipykernel_49116\3063883863.py:1: SyntaxWarning: invalid escape sequence '\s'
  corpus = ["F(x)={\sqrt{\frac{10x^{-8}}{-8}}}+C_{1} =-{\frac{5}{4x^{8}}}+C_{1}\quad{\mathrm {if~}}x<0.",
C:\Users\wjdrb\AppData\Local\Temp\ipykernel_49116\3063883863.py:2: SyntaxWarning: invalid escape sequence '\s'
  "F(x)=\sqrt{10}x^{-8/-8}+C_{1}=-\frac{5} {4}x^{8}+C_{1}\quad\mathrm{if~}x<0."]


In [7]:
from tqdm import tqdm
from tokenizers import Tokenizer, trainers, pre_tokenizers
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from huggingface_hub import HfApi, HfFolder
from typing import List

def train_and_upload_tokenizer(corpus: List[str], vocab_size: int = 30000, repo_name: str = "Kyudan/TeXBLUE-Tokenizer"):
    # 토크나이저와 트레이너 초기화
    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                         vocab_size=vocab_size)
    tokenizer.pre_tokenizer = Whitespace()

    # tqdm을 사용하여 진행 상태 표시
    print("Training tokenizer...")
    corpus_with_progress = tqdm(corpus, desc="Training Progress", total=len(corpus))
    tokenizer.train_from_iterator(corpus_with_progress, trainer)

    # 토크나이저를 파일로 저장
    tokenizer.save("tokenizer.json")
    
    # Hugging Face Hub에 업로드
    api = HfApi()
    token = HfFolder.get_token()
    
    api.upload_file(
        path_or_fileobj="tokenizer.json",
        path_in_repo="tokenizer.json",
        repo_id=repo_name,
        token=token,
    )
    
    print(f"Tokenizer uploaded to Hugging Face Hub under the repository {repo_name}")

In [8]:
train_and_upload_tokenizer(processed_corpus)

Training tokenizer...


Training Progress: 100%|██████████| 119194221/119194221 [02:42<00:00, 732722.75it/s]


Tokenizer uploaded to Hugging Face Hub under the repository Kyudan/TeXBLUE-Tokenizer


In [13]:
import re
from typing import List
import math
from collections import Counter
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import Tokenizer
from huggingface_hub import snapshot_download

def load_tokenizer_from_hub(repo_name: str = "Kyudan/TeXBLEU-Tokenizer") -> Tokenizer:
    # Hugging Face Hub에서 토크나이저 다운로드
        snapshot_dir = snapshot_download(repo_id=repo_name)
    
    # 다운로드한 토크나이저 불러오기
        tokenizer = Tokenizer.from_file(f"{snapshot_dir}/tokenizer.json")
        return tokenizer


class TeXBLEU:
    # List: 문자열들의 리스트.
    def __init__(self, repo_name: str):
        self.tokenizer = self._load_tokenizer_from_hub(repo_name)
    
    def _load_tokenizer_from_hub(self, repo_name: str) -> Tokenizer:
        return load_tokenizer_from_hub(repo_name)

    #스트링 형으로 이루어진 리스트와 int형 vocab size를 입력으로 받아서  Tokenizer 객체를 반환한다.
    def _train_bpe_tokenizer(self, corpus: List[str], vocab_size: int) -> Tokenizer:
        tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
        trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                             vocab_size=vocab_size)
        # tokenizer의 전처리 단계로 공백을 기준으로 분리하는 Whitespace() 전처리기를 설정.
        tokenizer.pre_tokenizer = Whitespace()
        tokenizer.train_from_iterator(corpus, trainer)
        return tokenizer
    
    # 명령어 앞에 한칸 띄고, 여러 공백은 지우기.
    def preprocess_latex(self, text: str) -> str:
        # Add space before '\'
        text = re.sub(r'(?<![\\])(\\)', r' \1', text)
        # Remove multiple spaces
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    """
    input:   tokens = ["\\frac", "{", "1", "}", "{", "2", "}"]

    output:           ["0:\\frac", "1:{", "2:1", "3:}", "4:{", "5:2", "6:}"]

    """
    def add_positional_encoding(self, tokens: List[str]) -> List[str]:
        return [f"{i}:{token}" for i, token in enumerate(tokens)]
    
    def calculate_texbleu(self, reference: str, candidate: str, max_n: int = 4) -> float:
        ref_tokens = self.tokenizer.encode(self.preprocess_latex(reference)).tokens
        cand_tokens = self.tokenizer.encode(self.preprocess_latex(candidate)).tokens
        print(f"ref_tokens : {ref_tokens}")
        print(f"cand_tokens : {cand_tokens}")
        # 아래 두줄을 주석으로 두면 positional encoding 끄게됨.
        ref_tokens = self.add_positional_encoding(ref_tokens)
        cand_tokens = self.add_positional_encoding(cand_tokens)
        
        bp = self._brevity_penalty(ref_tokens, cand_tokens)
        
        scores = []
        for n in range(1, max_n + 1):
            scores.append(self._modified_precision(ref_tokens, cand_tokens, n)) # n은 n-gram의 n임.
        
        if 0 in scores:
            return 0
        
        score = bp * math.exp(sum(math.log(s) for s in scores) / max_n) #modified_precision들의 평균을 exp취한 것에 bp 를 곱함.
        return score
    
    def _brevity_penalty(self, ref_tokens: List[str], cand_tokens: List[str]) -> float:
        r = len(ref_tokens)
        c = len(cand_tokens)
        
        if c > r:
            return 1
        else:
            return math.exp(1 - r/c)
    
    def _modified_precision(self, ref_tokens: List[str], cand_tokens: List[str], n: int) -> float:
        ref_ngrams = Counter(self._get_ngrams(ref_tokens, n))
        cand_ngrams = Counter(self._get_ngrams(cand_tokens, n))
        
        max_counts = {}
        for ngram, count in cand_ngrams.items():
            max_counts[ngram] = max(0, count - max(0, count - ref_ngrams[ngram]))
        
        if len(cand_ngrams) == 0:
            return 0
        
        return sum(max_counts.values()) / sum(cand_ngrams.values())
    
    """
    예시:
    입력: tokens = ["The", "quick", "brown", "fox"], n = 2
    출력: [("The", "quick"), ("quick", "brown"), ("brown", "fox")]
    """
    def _get_ngrams(self, tokens: List[str], n: int) -> List[tuple]:
        return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]



F(x)=\sqrt{10}x^{-8/-8}+C_{1}=-\frac{5} {4}x^{8}+C_{1}\quad\mathrm{if~}x<0.

$F(x)=\sqrt{10}x^{-8/-8}+C_{1}=-\frac{5} {4}x^{8}+C_{1}\quad\mathrm{if~}x<0.$

F(x)={\sqrt{\frac{10x^{-8}}{-8}}}+C_{1} =-{\frac{5}{4x^{8}}}+C_{1}\quad{\mathrm {if~}}x<0.

$F(x)={\sqrt{\frac{10x^{-8}}{-8}}}+C_{1} =-{\frac{5}{4x^{8}}}+C_{1}\quad{\mathrm {if~}}x<0.$

In [14]:
texbleu = TeXBLEU("Kyudan/TeXBLEU-Tokenizer")


reference = r"F(x)={\sqrt{\frac{10x^{-8}}{-8}}}+C_{1} =-{\frac{5}{4x^{8}}}+C_{1}\quad{\mathrm {if~}}x<0."
candidate1 = r"F(x)=\sqrt{10}x^{-8/-8}+C_{1}=-\frac{5} {4}x^{8}+C_{1}\quad\mathrm{if~}x<0."

score1 = texbleu.calculate_texbleu(reference, candidate1)
print(f"TeXBLEU score for candidate1: {score1:.4f}")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

ref_tokens : ['F', '(', 'x', ')=', '{', '\\', 'sqrt', '{', '\\', 'frac', '{', '10', 'x', '^{-', '8', '}}', '{-', '8', '}}}', '+', 'C_', '{', '1', '}', '=-', '{', '\\', 'frac', '{', '5', '}{', '4x', '^{', '8', '}}}', '+', 'C_', '{', '1', '}', '\\', 'quad', '{', '\\', 'mathrm', '{', 'if', '~', '}}', 'x', '<', '0', '.']
cand_tokens : ['F', '(', 'x', ')=', '\\', 'sqrt', '{', '10', '}', 'x', '^{-', '8', '/-', '8', '}+', 'C_', '{', '1', '}=-', '\\', 'frac', '{', '5', '}', '{', '4', '}', 'x', '^{', '8', '}+', 'C_', '{', '1', '}', '\\', 'quad', '\\', 'mathrm', '{', 'if', '~}', 'x', '<', '0', '.']
TeXBLEU score for candidate1: 0.0473
